In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [2]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize((64, 64)),
    ]
)

In [3]:
anime_data = datasets.MNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform,
)

In [4]:
data_loader = torch.utils.data.DataLoader(
    dataset=anime_data, batch_size=64, shuffle=True
)

In [14]:
class LinearAutoencoder(nn.Module):
    def __init__(self):
        super(LinearAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(64 * 64, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 12),
            nn.ReLU(),
            nn.Linear(12, 3),
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(),
            nn.Linear(12, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 64 * 64),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = LinearAutoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)

In [16]:
num_epochs = 320
outputs=[]
for epoch in range(num_epochs):
    for img, _ in data_loader:
        img = img.to(device)
        img = img.view(img.size(0), -1)
        recon = model(img)
        loss = criterion(recon, img)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch: {epoch+1}, Loss: {loss.item():.4f}")
    outputs.append((epoch, img, recon))

/home/bored/.local/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch: 1, Loss: 0.0576
Epoch: 2, Loss: 0.0582
Epoch: 3, Loss: 0.0588
Epoch: 4, Loss: 0.0605
Epoch: 5, Loss: 0.0630
Epoch: 6, Loss: 0.0578
Epoch: 7, Loss: 0.0526
Epoch: 8, Loss: 0.0597
Epoch: 9, Loss: 0.0554
Epoch: 10, Loss: 0.0606
Epoch: 11, Loss: 0.0605
Epoch: 12, Loss: 0.0550
Epoch: 13, Loss: 0.0565
Epoch: 14, Loss: 0.0565
Epoch: 15, Loss: 0.0645
Epoch: 16, Loss: 0.0562
Epoch: 17, Loss: 0.0573
Epoch: 18, Loss: 0.0538
Epoch: 19, Loss: 0.0585
Epoch: 20, Loss: 0.0581
Epoch: 21, Loss: 0.0626
Epoch: 22, Loss: 0.0585
Epoch: 23, Loss: 0.0635
Epoch: 24, Loss: 0.0571
Epoch: 25, Loss: 0.0577
Epoch: 26, Loss: 0.0550
Epoch: 27, Loss: 0.0497
Epoch: 28, Loss: 0.0531
Epoch: 29, Loss: 0.0635
Epoch: 30, Loss: 0.0569
Epoch: 31, Loss: 0.0517
Epoch: 32, Loss: 0.0545
Epoch: 33, Loss: 0.0577
Epoch: 34, Loss: 0.0586
Epoch: 35, Loss: 0.0597
Epoch: 36, Loss: 0.0543
Epoch: 37, Loss: 0.0602
Epoch: 38, Loss: 0.0577
Epoch: 39, Loss: 0.0572
Epoch: 40, Loss: 0.0593
Epoch: 41, Loss: 0.0553
Epoch: 42, Loss: 0.0589
E

In [18]:
torch.save(model.state_dict(), ``"./models/linear-auto-encoder.pth")

In [20]:
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = ConvAutoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)

In [24]:
import time
num_epochs = 320
outputs=[]
for epoch in range(num_epochs):
    for img, _ in data_loader:
        bruh = time.time()
        img = img.to(device)
        recon = model(img)
        loss = criterion(recon, img)
        waaa = time.time()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch: {epoch+1}, Loss: {loss.item():.4f}")
    outputs.append((epoch, img, recon))
    print(waaa-bruh)

Epoch: 1, Loss: 0.0062
0.0009338855743408203
Epoch: 2, Loss: 0.0053
0.0009436607360839844
Epoch: 3, Loss: 0.0050
0.0009496212005615234
Epoch: 4, Loss: 0.0055
0.0009670257568359375
Epoch: 5, Loss: 0.0051
0.0009510517120361328
Epoch: 6, Loss: 0.0053
0.0009949207305908203
Epoch: 7, Loss: 0.0050
0.0010416507720947266
Epoch: 8, Loss: 0.0046
0.0009200572967529297
Epoch: 9, Loss: 0.0055
0.000957489013671875
Epoch: 10, Loss: 0.0054
0.0009348392486572266
Epoch: 11, Loss: 0.0054
0.0009717941284179688
Epoch: 12, Loss: 0.0050
0.0009303092956542969
Epoch: 13, Loss: 0.0048
0.0009453296661376953
Epoch: 14, Loss: 0.0047
0.0009601116180419922
Epoch: 15, Loss: 0.0052
0.0009453296661376953
Epoch: 16, Loss: 0.0050
0.0009014606475830078
Epoch: 17, Loss: 0.0052
0.0009081363677978516
Epoch: 18, Loss: 0.0050
0.0009415149688720703
Epoch: 19, Loss: 0.0045
0.0009331703186035156
Epoch: 20, Loss: 0.0048
0.0009312629699707031
Epoch: 21, Loss: 0.0052
0.0009417533874511719
Epoch: 22, Loss: 0.0044
0.000904798507690429

In [25]:
torch.save(model.state_dict(), "./models/conv-auto-encoder.pth")

In [ ]:
model = ConvAutoencoder()
model.load_state_dict(torch.load("./models/conv-auto-encoder.pth"))
model.eval()